In [2]:
import pandas as pd
import duckdb

In [3]:
CPI25 = pd.read_csv('PCPI25M2.csv')
CPI25

,DATE,CPI
0,1947:01,21.5
1,1947:02,21.6
2,1947:03,22.0
3,1947:04,22.0
4,1947:05,22.0
...,...,...
932,2024:09,314.9
933,2024:10,315.6
934,2024:11,316.4
935,2024:12,317.6


In [ ]:
file = 'Fed_CPI2.db'

In [6]:
#trunc method
def trunc_and_load(con,data):
    # we simply truncate the table and load the new data
    con.sql(("CREATE OR REPLACE TABLE cpi_trunc AS SELECT * FROM data"))
with duckdb.connect(file) as con:
    trunc_and_load(con, CPI25)
    print(con.sql('SELECT * FROM cpi_trunc').fetchdf()) 



        DATE    CPI
0    1947:01   21.5
1    1947:02   21.6
2    1947:03   22.0
3    1947:04   22.0
4    1947:05   22.0
..       ...    ...
932  2024:09  314.9
933  2024:10  315.6
934  2024:11  316.4
935  2024:12  317.6
936  2025:01  319.1

[937 rows x 2 columns]


In [ ]:
#drop mistaken addition to db
with duckdb.connect(file) as con:
    con.sql('DROP TABLE IF EXISTS data_trunc')

In [7]:
CPI24 = pd.read_csv('PCPI24M1.csv')

In [18]:
def append_load(con, data):
    con.sql("""
        CREATE OR REPLACE TABLE cpi_append AS 
        SELECT *, CAST(REPLACE(DATE, ':', '') AS INTEGER) AS date_strip 
        FROM cpi_append
    """)
    
    # Get the max date_strip value
    max_date = con.sql('SELECT MAX(date_strip) FROM cpi_append').fetchdf().values[0][0]
    
    # Filter new data based on max_date
    data['date_strip'] = data['DATE'].str.replace(':', '').astype(int)
    data = data[data['date_strip'] > max_date]
    
    # Append new data to cpi_append
    con.sql("INSERT INTO cpi_append SELECT * FROM data")
    
with duckdb.connect(file) as con:
    con.sql('BEGIN TRANSACTION')
    append_load(con, CPI25)
    con.sql('COMMIT')
    print(con.sql('SELECT * FROM cpi_append').fetchdf())


        DATE    CPI  date_strip
0    1947:01  21.5       194701
1    1947:02  21.6       194702
2    1947:03  22.0       194703
3    1947:04  22.0       194704
4    1947:05  22.0       194705
..       ...    ...         ...
932  2024:09  314.9      202409
933  2024:10  315.6      202410
934  2024:11  316.4      202411
935  2024:12  317.6      202412
936  2025:01  319.1      202501

[937 rows x 3 columns]


In [ ]:
#inc method